In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from sklearn import tree

In [21]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
sample = pd.read_csv("sample.csv")

In [9]:
train

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [10]:
train.columns.difference(test.columns)

Index(['Loan_Status'], dtype='object')

0      1
1      1
2      1
3      0
4      1
      ..
609    1
610    1
611    1
612    1
613    1
Name: Education, Length: 614, dtype: int32

In [22]:
test['Source'] = 'test'
train['Source'] = 'train'
df = pd.concat([train, test])

In [75]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Source
0,LP001002,1.0,0,0.0,1,0.0,5849,0.0,142.51153,360.0,1.0,1,Y,train
1,LP001003,1.0,1,1.0,1,0.0,4583,1508.0,128.00000,360.0,1.0,-1,N,train
2,LP001005,1.0,1,0.0,1,1.0,3000,0.0,66.00000,360.0,1.0,1,Y,train
3,LP001006,1.0,1,0.0,0,0.0,2583,2358.0,120.00000,360.0,1.0,1,Y,train
4,LP001008,1.0,0,0.0,1,0.0,6000,0.0,141.00000,360.0,1.0,1,Y,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,1.0,1,3.0,0,1.0,4009,1777.0,113.00000,360.0,1.0,1,NaN,test
363,LP002975,1.0,1,0.0,1,0.0,4158,709.0,115.00000,360.0,1.0,1,NaN,test
364,LP002980,1.0,0,0.0,1,0.0,3250,1993.0,126.00000,360.0,-1.0,0,NaN,test
365,LP002986,1.0,1,0.0,1,0.0,5000,2393.0,158.00000,360.0,1.0,-1,NaN,test


In [24]:
df['Education'] = np.where(df['Education'] == 'Graduate', 1, 0)

In [34]:
df['Gender'].value_counts()

Male      775
Female    182
Name: Gender, dtype: int64

In [30]:
df['Gender'] = np.where(df['Gender'] == 'Graduate', 1, 0)

CoapplicantIncome
0.000000        419
16.120001         1
187.000000        1
189.000000        1
240.000000        1
               ... 
15000.000000      1
20000.000000      2
24000.000000      1
33837.000000      1
41667.000000      1
Name: Gender, Length: 437, dtype: int64

In [33]:
df.corr()

,Education,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
Education,1.000000,0.138909,0.060380,0.165053,0.024396,0.066182
ApplicantIncome,0.138909,1.000000,-0.114247,0.551811,-0.023089,0.023378
CoapplicantIncome,0.060380,-0.114247,1.000000,0.179228,-0.043860,-0.027253
LoanAmount,0.165053,0.551811,0.179228,1.000000,0.055636,-0.008235
Loan_Amount_Term,0.024396,-0.023089,-0.043860,0.055636,1.000000,-0.020439
Credit_History,0.066182,0.023378,-0.027253,-0.008235,-0.020439,1.000000


In [38]:
df['Gender'] = df['Gender'].replace({'Male': 1, 'Female' : 0})

TypeError: Cannot compare types 'ndarray(dtype=float64)' and 'str'

In [41]:
df['Gender'].value_counts()

 1.0    775
 0.0    182
-1.0     24
Name: Gender, dtype: int64

In [40]:
df['Gender'] = df['Gender'].transform(lambda x: x.fillna(-1))

In [69]:
df.isnull().sum()

Loan_ID                0
Gender                 0
Married                0
Dependents             0
Education              0
Self_Employed          0
ApplicantIncome        0
CoapplicantIncome      0
LoanAmount             0
Loan_Amount_Term       0
Credit_History         0
Property_Area          0
Loan_Status          367
Source                 0
dtype: int64

In [47]:
df['Married'].value_counts()

 1    631
 0    347
-1      3
Name: Married, dtype: int64

In [45]:
df['Married'] = df['Married'].transform(lambda x: x.fillna(-1))

In [46]:
df['Married'] = df['Married'].replace({'Yes' : 1 , 'No' : 0 })

In [54]:
df['Dependents'].value_counts()

 0.0    545
 2.0    160
 1.0    160
 3.0     91
-1.0     25
Name: Dependents, dtype: int64

In [50]:
df['Dependents'] = df['Dependents'].replace({'3+' : 3 , '1' : 1, '2' : 2 , '0' : 0})

In [53]:
df['Dependents'] = df['Dependents'].transform(lambda x: x.fillna(-1))

In [56]:
df['Self_Employed'] = df['Self_Employed'].replace({'Yes' : 1 , 'No' : 0 })
df['Self_Employed'] = df['Self_Employed'].transform(lambda x: x.fillna(-1))
df['Self_Employed'].value_counts()

 0.0    807
 1.0    119
-1.0     55
Name: Self_Employed, dtype: int64

In [64]:
df['Loan_Amount_Term'].value_counts()

360.000000    823
180.000000     66
480.000000     23
342.201873     20
300.000000     20
240.000000      8
84.000000       7
120.000000      4
36.000000       3
60.000000       3
12.000000       2
350.000000      1
6.000000        1
Name: Loan_Amount_Term, dtype: int64

In [60]:
df['LoanAmount'] = df['LoanAmount'].transform(lambda x: x.fillna((x.mean())))

In [63]:
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].transform(lambda x: x.fillna((x.mean())))

In [73]:
df['Property_Area'] = df['Property_Area'].replace({'Semiurban' : 0, 'Urban' : 1, 'Rural' : -1})

In [67]:
df['Credit_History'] = df['Credit_History'].transform(lambda x: x.fillna(-1))

In [70]:
scale_cols = df.columns[:-2].tolist()

In [71]:
scale_cols

['Loan_ID',
 'Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History',
 'Property_Area']

In [74]:
df['Property_Area'].value_counts()

 0    349
 1    342
-1    290
Name: Property_Area, dtype: int64

In [76]:
dfx = df.pop('Loan_ID')

In [77]:
dfx

0      LP001002
1      LP001003
2      LP001005
3      LP001006
4      LP001008
         ...   
362    LP002971
363    LP002975
364    LP002980
365    LP002986
366    LP002989
Name: Loan_ID, Length: 981, dtype: object

In [78]:
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Source
0,1.0,0,0.0,1,0.0,5849,0.0,142.51153,360.0,1.0,1,Y,train
1,1.0,1,1.0,1,0.0,4583,1508.0,128.00000,360.0,1.0,-1,N,train
2,1.0,1,0.0,1,1.0,3000,0.0,66.00000,360.0,1.0,1,Y,train
3,1.0,1,0.0,0,0.0,2583,2358.0,120.00000,360.0,1.0,1,Y,train
4,1.0,0,0.0,1,0.0,6000,0.0,141.00000,360.0,1.0,1,Y,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,1.0,1,3.0,0,1.0,4009,1777.0,113.00000,360.0,1.0,1,NaN,test
363,1.0,1,0.0,1,0.0,4158,709.0,115.00000,360.0,1.0,1,NaN,test
364,1.0,0,0.0,1,0.0,3250,1993.0,126.00000,360.0,-1.0,0,NaN,test
365,1.0,1,0.0,1,0.0,5000,2393.0,158.00000,360.0,1.0,-1,NaN,test


In [79]:
dftrain = df[df['Source'] ==  'train']
dftest = df[df['Source'] ==  'test']

In [80]:
dftrain

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Source
0,1.0,0,0.0,1,0.0,5849,0.0,142.51153,360.0,1.0,1,Y,train
1,1.0,1,1.0,1,0.0,4583,1508.0,128.00000,360.0,1.0,-1,N,train
2,1.0,1,0.0,1,1.0,3000,0.0,66.00000,360.0,1.0,1,Y,train
3,1.0,1,0.0,0,0.0,2583,2358.0,120.00000,360.0,1.0,1,Y,train
4,1.0,0,0.0,1,0.0,6000,0.0,141.00000,360.0,1.0,1,Y,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0,0.0,1,0.0,2900,0.0,71.00000,360.0,1.0,-1,Y,train
610,1.0,1,3.0,1,0.0,4106,0.0,40.00000,180.0,1.0,-1,Y,train
611,1.0,1,1.0,1,0.0,8072,240.0,253.00000,360.0,1.0,1,Y,train
612,1.0,1,2.0,1,0.0,7583,0.0,187.00000,360.0,1.0,1,Y,train


In [83]:
del dftrain['Source']
del dftest['Source']

In [85]:
dfy = dftrain.pop('Loan_Status')

In [87]:
del dftest['Loan_Status']

In [90]:
dftrain.shape 


(614, 11)

In [94]:
scalar = StandardScaler()
trainscale = scalar.fit_transform(dftrain)
testscale  = scalar.fit_transform(dftest)

In [98]:
testscale 

array([[ 0.50359765,  0.75835829, -0.71080969, ...,  0.27068762,
         0.52045769,  1.11876379],
       [ 0.50359765,  0.75835829,  0.20131323, ...,  0.27068762,
         0.52045769,  1.11876379],
       [ 0.50359765,  0.75835829,  1.11343616, ...,  0.27068762,
         0.52045769,  1.11876379],
       ...,
       [ 0.50359765, -1.31863792, -0.71080969, ...,  0.27068762,
        -2.74463585, -0.09598861],
       [ 0.50359765,  0.75835829, -0.71080969, ...,  0.27068762,
         0.52045769, -1.31074101],
       [ 0.50359765, -1.31863792, -0.71080969, ..., -2.51861407,
         0.52045769, -1.31074101]])

In [109]:
clf = GaussianNB()
clf.fit(trainscale, dfy)
NBpred = clf.predict(testscale)
NBtrain = clf.predict(trainscale)

In [128]:
KNN = KNeighborsClassifier(n_neighbors = 1, metric = 'manhattan', weights = 'distance', n_jobs = -1)

In [129]:
KNN.fit(trainscale, dfy)
KNpred = KNN.predict(testscale)
KNtrain = KNN.predict(trainscale)

In [130]:
accuracy_score(dfy, KNtrain)

1.0

In [108]:
print(confusion_matrix(dfy, KNtrain))
print(classification_report(dfy, KNtrain))

[[192   0]
 [  0 422]]
              precision    recall  f1-score   support

           N       1.00      1.00      1.00       192
           Y       1.00      1.00      1.00       422

    accuracy                           1.00       614
   macro avg       1.00      1.00      1.00       614
weighted avg       1.00      1.00      1.00       614



In [110]:
accuracy_score(dfy, NBtrain)

0.6612377850162866

In [111]:
sample.shape

(367, 2)

In [112]:
sample

,Loan_ID,Loan_Status
0,LP001015,N
1,LP001022,N
2,LP001031,N
3,LP001035,N
4,LP001051,N
...,...,...
362,LP002971,N
363,LP002975,N
364,LP002980,N
365,LP002986,N


In [134]:
sample['Loan_Status'] = Mnpred

In [135]:
sample

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,N
4,LP001051,Y
...,...,...
362,LP002971,Y
363,LP002975,Y
364,LP002980,N
365,LP002986,Y


In [136]:
sample.to_csv('Final1.csv')

In [120]:
lg = LogisticRegression()
lg.fit(trainscale, dfy)
lgpred = lg.predict(testscale)
lgtrain = lg.predict(trainscale)

In [121]:
accuracy_score(dfy, lgtrain)

0.6856677524429967

In [132]:
Mn = BernoulliNB()
Mn.fit(trainscale, dfy)
Mnpred = Mn.predict(testscale)
Mntrain = Mn.predict(trainscale)

In [133]:
accuracy_score(dfy, Mntrain)

0.7703583061889251

In [138]:
dt = tree.DecisionTreeClassifier()
dt.fit(trainscale, dfy)
dtpred = dt.predict(testscale)
dttrain = dt.predict(trainscale)
accuracy_score(dfy, dttrain)

1.0

In [139]:
sample['Loan_Status'] = dtpred
sample.to_csv('Final2.csv')